# Imports

Here we import the required libraries

In [1]:
'''%pip install torchsummary albumentations wandb --quiet
%pip install tqdm 
%pip install pandas 
%pip install torch
%pip install PIL
%pip install torchvision
%pip install matplotlib
%pip install scikit-learn
%pip install scikit-image
%pip install setuptools
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install optuna '''


'%pip install torchsummary albumentations wandb --quiet\n%pip install tqdm \n%pip install pandas \n%pip install torch\n%pip install PIL\n%pip install torchvision\n%pip install matplotlib\n%pip install scikit-learn\n%pip install scikit-image\n%pip install setuptools\n%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118\n%pip install optuna '

In [2]:
from __future__ import print_function, division
import copy
import math
import os
import os.path as osp
import time
import typing as ty
from datetime import datetime
from functools import reduce
from itertools import chain, islice
from multiprocessing import cpu_count

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import matplotlib.pyplot as plt
import numpy as np
from numpy.typing import NDArray
import optuna
import pandas as pd
from PIL import Image, UnidentifiedImageError
from skimage import io, transform
from sklearn.model_selection import train_test_split
import torch
from torch import nn, Tensor
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import Optimizer
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.ops import box_iou
from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_notebook
import wandb

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

c:\Users\allc0363\AppData\Local\Programs\Python\Python313\Lib\site-packages\albumentations\__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.7' (you have '2.0.6'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
c:\Users\allc0363\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device}')

Using cuda


In [4]:
# Crear un generador para el DataLoader
generator = torch.Generator()
generator.manual_seed(42)

## Dataset

In [5]:
DATA_DIR = '../Datos/' 
WORK_DIR = '../Datos/' 
DIR_Results = './Resultados/'
BATCH_SIZE = 32

IMG_ORIG_H, IMG_ORIG_W = 720, 1280
TARGET_H, TARGET_W = 224, 224 

img_dir = osp.join(DATA_DIR, "images/images")
df = pd.read_csv(osp.join(DATA_DIR, "train.csv"))

obj2id  = {'f16':0,'cougar':1,'chinook':2,'ah64':3,'f15':4,'seahawk':5}
#obj2id = {'f16':0, 'cougar':1, 'chinook':2, 'ah64':3, 'f15':4, 'seahawk':5}
id2obj = {v: k for k, v in obj2id.items()} 

NUM_CLASSES = len(obj2id)
MEANS_IMAGENET = [0.485, 0.456, 0.406] 
STDS_IMAGENET = [0.229, 0.224, 0.225]  

df["class_id"] = df["class"].map(obj2id)
columns_f=['filename','xmin','ymin','xmax','ymax','class','class_id']
df= df[columns_f].copy()

In [6]:
print(f"Global: TARGET_H={TARGET_H}, TARGET_W={TARGET_W}, NUM_CLASSES={NUM_CLASSES}")
print(f"Global: IMG_ORIG_H={IMG_ORIG_H}, IMG_ORIG_W={IMG_ORIG_W}")

Global: TARGET_H=224, TARGET_W=224, NUM_CLASSES=6
Global: IMG_ORIG_H=720, IMG_ORIG_W=1280


## EDA

In [7]:
img_filename = osp.join(DATA_DIR, "images/images",'image_00077.jpeg')

img1 = cv2.imread(img_filename)
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
img2 = io.imread(img_filename) 

In [8]:
print(img1.shape)
print(img1.transpose((2,0,1)).shape)

(720, 1280, 3)
(3, 720, 1280)


In [9]:
list_image = list(df.filename)
data_shape = []
data_dim = []
data_w = []
data_h = []

for i in tqdm(list_image): ## tqdm(list_image)dura 40 segundos
    ruta_imagen = osp.join(img_dir, i)
    imagen = io.imread(ruta_imagen)
    shapes = imagen.shape
    dimen = imagen.ndim
    imagen = Image.open(ruta_imagen)
    w, h = imagen.size
    
    data_w.append(w)
    data_h.append(h)
    data_shape.append(shapes)
    data_dim.append(dimen)

data_w_h = pd.DataFrame([list_image,data_shape,data_dim,data_w,data_h]).T.rename(columns={0:'filename',1:'shapes',2:'ndim',3:'w',4:'h'}) 

100%|██████████| 189/189 [00:01<00:00, 131.40it/s]


In [10]:
data_w_h['w'].value_counts()

w
1280    189
Name: count, dtype: int64

In [11]:
data_w_h['ndim'].value_counts()


ndim
3    189
Name: count, dtype: int64

In [12]:
data_w_h['shapes'].value_counts()

shapes
(720, 1280, 3)    189
Name: count, dtype: int64

In [13]:
df['class_id'].value_counts()

class_id
0    41
1    37
2    35
3    34
4    23
5    19
Name: count, dtype: int64

In [14]:
df['class'].value_counts()

class
f16        41
cougar     37
chinook    35
ah64       34
f15        23
seahawk    19
Name: count, dtype: int64

In [15]:
df[df['xmin']>=df['xmax']].shape, df[df['ymin']>=df['ymax']].shape

((0, 7), (0, 7))

## Normalizamos los bboxes (En la siguiente monitoria hablaremos de la importancia)

In [16]:
print(df[["ymin", "ymax", "xmin", "xmax"]].describe())

             ymin        ymax        xmin         xmax
count  189.000000  189.000000  189.000000   189.000000
mean   186.396825  425.455026  401.238095   902.666667
std    112.079883  103.348946  232.361996   213.588688
min      1.000000  154.000000    1.000000   298.000000
25%    109.000000  359.000000  210.000000   730.000000
50%    188.000000  417.000000  418.000000   901.000000
75%    267.000000  494.000000  572.000000  1071.000000
max    440.000000  702.000000  944.000000  1280.000000


In [17]:
# Normalización de Bounding Boxes.
# Las coordenadas se convierten a relativas [0,1].
df[["ymin", "ymax"]] = df[["ymin", "ymax"]].div(IMG_ORIG_H, axis=0)
df[["xmin", "xmax"]] = df[["xmin", "xmax"]].div(IMG_ORIG_W, axis=0)

In [18]:
# Asegurar que los bboxes estén acotados a [0,1] después de la normalización
df[['xmin', 'ymin', 'xmax', 'ymax']] = df[['xmin', 'ymin', 'xmax', 'ymax']].clip(0.0, 1.0)


In [19]:
print(df[["ymin", "ymax", "xmin", "xmax"]].describe())

             ymin        ymax        xmin        xmax
count  189.000000  189.000000  189.000000  189.000000
mean     0.258884    0.590910    0.313467    0.705208
std      0.155667    0.143540    0.181533    0.166866
min      0.001389    0.213889    0.000781    0.232813
25%      0.151389    0.498611    0.164062    0.570312
50%      0.261111    0.579167    0.326562    0.703906
75%      0.370833    0.686111    0.446875    0.836719
max      0.611111    0.975000    0.737500    1.000000


## Particionamiento

In [20]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['class_id'])


In [21]:
train_df['class'].value_counts(1) * 100

class
f16        21.854305
cougar     19.867550
chinook    18.543046
ah64       17.880795
f15        11.920530
seahawk     9.933775
Name: proportion, dtype: float64

In [22]:
val_df['class_id'].value_counts(1) * 100

class_id
0    21.052632
3    18.421053
2    18.421053
1    18.421053
4    13.157895
5    10.526316
Name: proportion, dtype: float64

In [23]:
# --- 0. Funciones y Clases Auxiliares ---
def reset_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    # torch.backends.cudnn.deterministic = True
    # torch.backends.cudnn.benchmark = False

In [24]:
import torch

# DEFINICIÓN DE collate_fn_skip_none 
def collate_fn_skip_none(batch):
    batch_original_len = len(batch)
    batch_filtered = [item for item in batch if item is not None]
    
    if batch_original_len != len(batch_filtered) and batch_original_len > 0 :
        # print(f"DEBUG collate_fn: Original batch len: {batch_original_len}, Filtered batch len: {len(batch_filtered)}. {batch_original_len - len(batch_filtered)} muestra(s) eran None.")
        pass

    if not batch_filtered:
        # print("DEBUG collate_fn: Entire batch is None after filtering.")
        return None
    
    try:
        return torch.utils.data.dataloader.default_collate(batch_filtered)
    except Exception as e:
        # print(f"ERROR en default_collate dentro de collate_fn_skip_none: {e}")
        raise

In [25]:
# Definiciones de Clases de Transformación
class ToTensorDict:
    def __call__(self, sample):
        image, bbox, class_id = sample['image'], sample['bbox'], sample['class_id']
        image = image.transpose((2, 0, 1))
        sample['image'] = torch.from_numpy(image.copy()).float().div(255.0)
        sample['bbox'] = bbox 
        sample['class_id'] = class_id
        return sample

class NormalizeDict:
    def __init__(self, mean, std):
        self.mean = torch.tensor(mean, dtype=torch.float32).view(3, 1, 1)
        self.std = torch.tensor(std, dtype=torch.float32).view(3, 1, 1)

    def __call__(self, sample):
        sample['image'] = (sample['image'] - self.mean) / self.std
        return sample

## Clase para estructura de los datos

In [26]:
class militarDataset(Dataset):
    def __init__(self, df, img_dir, transform=None, labeled=True): 
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
        self.labeled = labeled 
        print(f"DEBUG militarDataset: Inicializado con {len(self.df)} filas. Transformaciones: {'Sí' if self.transform else 'No'}. Labeled: {self.labeled}")

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        if idx >= len(self.df):
            print(f"DEBUG militarDataset: Índice {idx} fuera de rango para DataFrame (len {len(self.df)}). Devolviendo None.")
            return None

        row = self.df.iloc[idx]
        img_filename = row.get('filename', f"DESCONOCIDO_idx_{idx}")
        
        try:
            img_path = os.path.join(self.img_dir, img_filename)
            if not osp.exists(img_path):
                print(f"DEBUG militarDataset: RUTA DE IMAGEN NO EXISTE {img_path} para {img_filename}. Saltando.")
                return None
            pil_image = Image.open(img_path).convert('RGB')
            numpy_image = np.array(pil_image)
        except FileNotFoundError:
            print(f"DEBUG militarDataset: FileNotFoundError para {img_path} ({img_filename}). Saltando.")
            return None
        except UnidentifiedImageError:
            print(f"DEBUG militarDataset: UnidentifiedImageError para {img_path} ({img_filename}). Saltando.")
            return None
        except Exception as e:
            print(f"DEBUG militarDataset: Error cargando imagen {img_filename} ({img_path}): {e}. Saltando.")
            return None
        
        # --- SI ES PARA TEST (NO ETIQUETADO) ---
        if not self.labeled:
            sample_to_transform = {'image': numpy_image}
            if self.transform:
                try:
                    transformed = self.transform(**sample_to_transform)
                    image_tensor = transformed['image']
                    return image_tensor # 📌 Devolver solo la imagen transformada
                except Exception as e:
                    print(f"DEBUG militarDataset: Error aplicando transformación (test) para {img_filename} (idx {idx}): {e}. Saltando.")
                    return None
            else: # Si no hay transformaciones (aunque para test deberías al menos tener Resize, Normalize, ToTensorV2)
                return torch.from_numpy(numpy_image.transpose((2, 0, 1))).float().div(255.0)

        # --- SI ES PARA ENTRENAMIENTO/VALIDACIÓN (ETIQUETADO) ---
        try:
            bbox_coords = row[['xmin', 'ymin', 'xmax', 'ymax']].values.astype(np.float32)
            class_id_val = int(row['class_id'])
        except KeyError as e:
            print(f"DEBUG militarDataset: Falta columna {e} para {img_filename} (idx {idx}) aunque labeled=True. Saltando.")
            return None
        except ValueError as e:
            print(f"DEBUG militarDataset: Valor inválido en bbox o class_id para {img_filename} (idx {idx}): {e}. Saltando.")
            return None

        
        if not (0 <= class_id_val < NUM_CLASSES):
            print(f"DEBUG militarDataset: class_id {class_id_val} fuera de rango [0, {NUM_CLASSES-1}] para {img_filename} (idx {idx}). Saltando.")
            return None

        sample_to_transform = {
            'image': numpy_image,
            'bboxes': [bbox_coords.tolist()], 
            'class_labels': [class_id_val] 
        }

        image_tensor, class_id_tensor, bbox_tensor = None, None, None

        if self.transform:
            try:
                transformed = self.transform(**sample_to_transform)
                
                img_from_transform = transformed.get('image')
                bboxes_from_transform = transformed.get('bboxes')
                class_labels_from_transform = transformed.get('class_labels')

                if img_from_transform is None:
                    print(f"DEBUG militarDataset: 'image' es None post-transformación para {img_filename} (idx {idx}). Saltando.")
                    return None
                image_tensor = img_from_transform

                if bboxes_from_transform and len(bboxes_from_transform) > 0:
                    current_bbox = bboxes_from_transform[0]
                    if isinstance(current_bbox, (list, tuple, np.ndarray)) and len(current_bbox) == 4:
                        bbox_tensor = torch.tensor(current_bbox, dtype=torch.float32).unsqueeze(0)
                        
                        if class_labels_from_transform and len(class_labels_from_transform) > 0:
                            _class_id_item = class_labels_from_transform[0]
                            if not (0 <= _class_id_item < NUM_CLASSES):
                                print(f"DEBUG militarDataset: class_id {_class_id_item} post-transform fuera de rango para {img_filename} (idx {idx}). Saltando.")
                                return None
                            class_id_tensor = torch.tensor(_class_id_item, dtype=torch.long)
                        else:
                            print(f"DEBUG militarDataset: class_labels perdidas post-transformación para {img_filename} (idx {idx}). Saltando.")
                            return None 
                    else:
                        print(f"DEBUG militarDataset: Formato de bbox inesperado para {img_filename} (idx {idx}) post-transformación. Saltando.")
                        return None
                else: 
                    print(f"DEBUG militarDataset: Bbox para {img_filename} (idx {idx}) removido o vacío post-transformación. Saltando.")
                    return None
                
                if not (isinstance(image_tensor, torch.Tensor) and 
                        isinstance(class_id_tensor, torch.Tensor) and 
                        isinstance(bbox_tensor, torch.Tensor)):
                    print(f"DEBUG militarDataset: Componente no es Tensor para {img_filename} (idx {idx}) post-transform. Saltando.")
                    return None
            except Exception as e:
                print(f"DEBUG militarDataset: Error aplicando/procesando transformación (train/val) para {img_filename} (idx {idx}): {e}. Saltando.")
                return None
        else: 
            if not (0 <= class_id_val < NUM_CLASSES):
                print(f"DEBUG militarDataset: class_id {class_id_val} (sin transform) fuera de rango para {img_filename} (idx {idx}). Saltando.")
                return None
            image_tensor = torch.from_numpy(numpy_image.transpose((2, 0, 1))).float().div(255.0)
            bbox_tensor = torch.tensor(bbox_coords, dtype=torch.float32).unsqueeze(0)
            class_id_tensor = torch.tensor(class_id_val, dtype=torch.long)
        
        return image_tensor, class_id_tensor, bbox_tensor

## Normalización (Ahora de los píxeles, es diferente a la normalización anterior)

In [27]:
# Definir medias y desviaciones estándar
MEANS_IMAGENET = [0.485, 0.456, 0.406]
STDS_IMAGENET = [0.229, 0.224, 0.225]
print(f"Usando medias de ImageNet: {MEANS_IMAGENET}")
print(f"Usando desviaciones estándar de ImageNet: {STDS_IMAGENET}")

Usando medias de ImageNet: [0.485, 0.456, 0.406]
Usando desviaciones estándar de ImageNet: [0.229, 0.224, 0.225]


## Transformaciones

In [28]:
train_transforms = A.Compose([
    A.Resize(height=TARGET_H, width=TARGET_W),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.2), 
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5, border_mode=cv2.BORDER_CONSTANT, value = 0), 
    # A.CoarseDropout(max_holes=8, max_height=TARGET_H//10, max_width=TARGET_W//10, min_holes=1, min_height=TARGET_H//20, min_width=TARGET_W//20, fill_value=0, p=0.3),
    # Normalización estándar de ResNet. max_pixel_value=255.0 asegura la división por 255.
    A.Normalize(mean=MEANS_IMAGENET, std=STDS_IMAGENET, max_pixel_value=255.0),
    ToTensorV2() # Convierte imagen a tensor PyTorch (C, H, W). No afecta bboxes.
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels'], min_area=16, min_visibility=0.2)) # pascal_voc: [xmin, ymin, xmax, ymax]

eval_transforms = A.Compose([
    A.Resize(height=TARGET_H, width=TARGET_W),
    A.Normalize(mean=MEANS_IMAGENET, std=STDS_IMAGENET, max_pixel_value=255.0),
    ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels'])) 



c:\Users\allc0363\AppData\Local\Programs\Python\Python313\Lib\site-packages\albumentations\core\validation.py:111: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
C:\Users\allc0363\AppData\Local\Temp\ipykernel_34400\3914898059.py:6: UserWarning: Argument(s) 'value' are not valid for transform ShiftScaleRotate
  A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5, border_mode=cv2.BORDER_CONSTANT, value = 0),


In [29]:
# train_transforms y eval_transforms globales

print("DEBUG: Definiendo transformaciones globales train_transforms y eval_transforms con pipeline MÍNIMO.")

# --- 📌 MODIFICACIÓN DRÁSTICA DE train_transforms (global) ---
train_transforms = A.Compose([
    A.Resize(height=TARGET_H, width=TARGET_W), # Necesario
    # --- Temporalmente SIN otras aumentaciones ---
    # A.HorizontalFlip(p=0.5),
    # A.VerticalFlip(p=0.2),
    # A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.3), # Intensidad reducida
    # A.ShiftScaleRotate(shift_limit=0.03, scale_limit=0.03, rotate_limit=10, p=0.3, 
    #                    border_mode=cv2.BORDER_CONSTANT), # 'value' eliminado o comentada la línea entera si da warning
    
    A.Normalize(mean=MEANS_IMAGENET, std=STDS_IMAGENET, max_pixel_value=255.0), # Necesario
    ToTensorV2() # Necesario
], bbox_params=A.BboxParams(format='pascal_voc', 
                             label_fields=['class_labels'], 
                             min_area=1,       # 📌 VALOR MÁS PERMISIVO POSIBLE
                             min_visibility=0.0 # 📌 VALOR MÁS PERMISIVO POSIBLE
                            ))


eval_transforms = A.Compose([ 
    A.Resize(height=TARGET_H, width=TARGET_W),
    A.Normalize(mean=MEANS_IMAGENET, std=STDS_IMAGENET, max_pixel_value=255.0),
    ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))


print("DEBUG: Creando train_dataset y val_dataset GLOBALES con las transformaciones simplificadas.")

if 'train_df' in locals() and not train_df.empty: # Comprueba si train_df existe y no está vacío
    train_dataset = militarDataset(df=train_df, img_dir=img_dir, transform=train_transforms)
    print(f"DEBUG: train_dataset global creado con {len(train_dataset)} items.")
else:
    print("Advertencia: train_df global no está definido o está vacío. No se creó train_dataset global.")
    train_dataset = [] # o None

if 'val_df' in locals() and not val_df.empty: # Comprueba si val_df existe y no está vacío
    val_dataset = militarDataset(df=val_df, img_dir=img_dir, transform=eval_transforms)
    print(f"DEBUG: val_dataset global creado con {len(val_dataset)} items.")
else:
    print("Advertencia: val_df global no está definido o está vacío. No se creó val_dataset global.")
    val_dataset = [] # o None

DEBUG: Definiendo transformaciones globales train_transforms y eval_transforms con pipeline MÍNIMO.
DEBUG: Creando train_dataset y val_dataset GLOBALES con las transformaciones simplificadas.
DEBUG militarDataset: Inicializado con 151 filas. Transformaciones: Sí. Labeled: True
DEBUG: train_dataset global creado con 151 items.
DEBUG militarDataset: Inicializado con 38 filas. Transformaciones: Sí. Labeled: True
DEBUG: val_dataset global creado con 38 items.


## Transfer Learning

In [30]:
class ResNet50MultiTaskModel(nn.Module):
    def __init__(self, num_classes=6, dropout=0.5, freeze_backbone=True, use_deeper_bbox_head=False): # Nuevo argumento
        super(ResNet50MultiTaskModel, self).__init__()

        # Cargar el modelo base ResNet50 pre-entrenado
        base_model = resnet50(weights=ResNet50_Weights.DEFAULT) #

        # Congelar los pesos del backbone si se especifica
        if freeze_backbone: #
            for param in base_model.parameters(): #
                param.requires_grad = False #

        # Extractor de características (todas las capas de ResNet excepto avgpool y fc)
        self.feature_extractor = nn.Sequential(*list(base_model.children())[:-2]) #
        
        # Pooling adaptativo para obtener un tamaño de salida fijo
        self.pooling = nn.AdaptiveAvgPool2d((1, 1)) #
        
        # Capa para aplanar las características
        self.flatten = nn.Flatten() #

        # --- Cabeza de Clasificación ---
        self.classification_dropout = nn.Dropout(dropout) #
        # La capa de entrada para el clasificador es 2048 (salida de ResNet50 antes de avgpool)
        self.classifier = nn.Linear(base_model.fc.in_features, num_classes) #

        # --- Cabeza de Regresión de Bounding Box ---
        self.bbox_dropout = nn.Dropout(dropout) #
        
        if use_deeper_bbox_head:
            print("Usando cabeza de regresión de BBox más profunda.")
            self.bbox_regressor = nn.Sequential(
                nn.Linear(base_model.fc.in_features, 1024), # base_model.fc.in_features es 2048
                nn.ReLU(),
                nn.Dropout(dropout), # Dropout adicional dentro de la cabeza
                nn.Linear(1024, 256),
                nn.ReLU(),
                nn.Linear(256, 4) # Salida de 4 coordenadas para el bbox
            )
            # Inicialización de pesos para la cabeza más profunda (opcional pero puede ayudar)
            for m in self.bbox_regressor.modules():
                if isinstance(m, nn.Linear):
                    nn.init.xavier_uniform_(m.weight)
                    if m.bias is not None:
                        nn.init.zeros_(m.bias)
        else:
            print("Usando cabeza de regresión de BBox simple (Linear).")
            self.bbox_regressor = nn.Linear(base_model.fc.in_features, 4) #
            # Inicialización de pesos para la cabeza simple (opcional)
            nn.init.xavier_uniform_(self.bbox_regressor.weight)
            # if self.bbox_regressor.bias is not None:
            #     nn.init.zeros_(self.bbox_regressor.bias)


    def forward(self, x: Tensor) -> tuple[Tensor, Tensor]:
        features = self.feature_extractor(x) #
        pooled_features = self.pooling(features) #
        flattened_features = self.flatten(pooled_features) #

        # Clasificación
        class_features_dropout = self.classification_dropout(flattened_features) #
        class_output = self.classifier(class_features_dropout) #

        # Regresión de Bounding Box
        bbox_features_dropout = self.bbox_dropout(flattened_features) #
        bbox_output = self.bbox_regressor(bbox_features_dropout) #
        
        return class_output, bbox_output

## Metricas

In [31]:
def iou_metric(y_true_bbox: torch.Tensor, y_pred_bbox: torch.Tensor, apply_sigmoid_to_pred=False):
    
    if y_true_bbox.device != y_pred_bbox.device:
        y_true_bbox = y_true_bbox.to(y_pred_bbox.device)
    if apply_sigmoid_to_pred:
         y_pred_bbox = torch.sigmoid(y_pred_bbox)
    if y_true_bbox.ndim == 3 and y_true_bbox.shape[1] == 1:
        y_true_bbox = y_true_bbox.squeeze(1)
    if y_pred_bbox.ndim == 3 and y_pred_bbox.shape[1] == 1:
        y_pred_bbox = y_pred_bbox.squeeze(1)
    if y_true_bbox.shape[0] == 0 or y_pred_bbox.shape[0] == 0:
        return torch.tensor(0.0, device=y_pred_bbox.device)
    pairwise_iou_matrix = box_iou(y_true_bbox, y_pred_bbox)
    if pairwise_iou_matrix.shape[0] == pairwise_iou_matrix.shape[1]:
        return torch.diag(pairwise_iou_matrix).mean()
    elif pairwise_iou_matrix.numel() > 0:
        return pairwise_iou_matrix.mean()
    else:
        return torch.tensor(0.0, device=y_pred_bbox.device)

In [32]:
def accuracy_metric(y_pred_class: torch.Tensor, y_true_class: torch.Tensor):
    """
    Calcula la precisión de la clasificación.

    Args:
        y_pred_class (torch.Tensor): Logits o probabilidades predichas por el modelo.
                                     Shape: (batch_size, num_classes)
        y_true_class (torch.Tensor): Etiquetas de clase verdaderas.
                                     Shape: (batch_size,) o (batch_size, 1)
    Returns:
        torch.Tensor: Precisión.
    """
    # Obtener las clases predichas tomando el argmax
    pred_labels = torch.argmax(y_pred_class, dim=1)

    # Asegurar que y_true_class tenga la misma forma que pred_labels
    if y_true_class.ndim == 2 and y_true_class.shape[1] == 1:
        y_true_class = y_true_class.squeeze(1)

    # Comparar con las etiquetas verdaderas
    correct = torch.eq(pred_labels, y_true_class).float() # Convertir a float para la suma
    total = torch.ones_like(correct) # Un tensor de unos con la misma forma

    # Calcular la precisión
    acc = torch.sum(correct) / torch.sum(total) 
    # result = torch.divide(torch.sum(correct), torch.sum(total)) 
    return acc

## Loss fn

In [33]:
def loss_fn_multitask(class_preds: torch.Tensor, bbox_preds: torch.Tensor,
                      true_classes: torch.Tensor, true_bboxes: torch.Tensor,
                      alpha: float = 0.5):
    """
    Calcula la pérdida combinada para clasificación y regresión de bounding box.

    Args:
        class_preds (torch.Tensor): Predicciones de clase del modelo (logits).
                                     Shape: (batch_size, num_classes)
        bbox_preds (torch.Tensor): Predicciones de bounding box del modelo.
                                   Shape: (batch_size, 4)
        true_classes (torch.Tensor): Etiquetas de clase verdaderas.
                                     Shape: (batch_size,) o (batch_size, 1) - deben ser long ints.
        true_bboxes (torch.Tensor): Coordenadas de bounding box verdaderas.
                                    Shape: (batch_size, 4)
        alpha (float): Peso para balancear la pérdida de regresión.
                       total_loss = (1-alpha)*cls_loss + alpha*reg_loss
    Returns:
        dict: Un diccionario con 'total_loss', 'cls_loss', 'reg_loss'.
    """
    # Asegurarse de que true_classes tenga el tipo de dato correcto para CrossEntropyLoss
    true_classes = true_classes.long()
    if true_classes.ndim == 2 and true_classes.shape[1] == 1:
        true_classes = true_classes.squeeze(1) # (batch_size, 1) -> (batch_size,)

    # Pérdida de Clasificación
    # class_preds son logits, true_classes son índices de clase
    cls_loss = F.cross_entropy(class_preds, true_classes)

    # Pérdida de Regresión para Bounding Box
    # bbox_preds y true_bboxes deben tener la misma forma [N, 4]
    reg_loss = F.mse_loss(bbox_preds, true_bboxes)
    # Alternativa: reg_loss = F.l1_loss(bbox_preds, true_bboxes)

    # Pérdida Total Combinada
    total_loss = (1 - alpha) * cls_loss + alpha * reg_loss

    return {
        'total_loss': total_loss,
        'reg_loss': reg_loss,
        'cls_loss': cls_loss
    }


## Callbacks

In [34]:
def printer_callback(logs: dict): # ty.Dict[str, ty.Any] si usas type hints
    """
    Imprime los logs del entrenamiento.
    """
    # print every 10 steps - asumiendo que 'iters' está en logs
    if 'iters' in logs and logs['iters'] % 10 != 0: # Cambié == a != para imprimir cada 10
        if logs['iters'] != 1: # Imprimir siempre la primera iteración
            return

    print(f"Iteración #: {logs.get('iters', 'N/A')}") # Usar .get para evitar KeyError
    for name, value in logs.items():
        if name == 'iters':
            continue

        if isinstance(value, (float, int)): # type(value) in [float, int] es menos idiomático
            value_str = f"{value:.4f}" if isinstance(value, float) else str(value)
        elif isinstance(value, torch.Tensor):
            # value_str = f"{value.item():.4f}" if value.numel() == 1 else str(value)
            value_str = f"{torch.round(value, decimals=4).item() if value.numel() == 1 else str(value.round(decimals=4))}"
        else:
            value_str = str(value)
        print(f"\t{name} = {value_str}")
    print("-" * 30) # Separador



## Training loop

In [35]:
import torch
import time # Para medir la duración de las épocas
import copy # Para guardar el mejor estado del modelo
import os   # Para la modificación de guardado del modelo


def train_multitask_model(
    model, train_loader, val_loader, classification_criterion, bbox_criterion,
    optimizer, device, num_epochs=30, bbox_loss_weight=1.0, patience=5,
    model_save_path="best_multitask_model.pth", apply_sigmoid_to_bbox_pred=False,
    scheduler=None # 
):
    # reset_seed(42) # Puedes llamarlo fuera o dentro si quieres resetear en cada llamada.

    history = {
        'train_loss_cls': [], 'train_loss_bbox': [], 'train_loss_total': [],
        'train_acc': [], 'train_iou': [],
        'val_loss_cls': [], 'val_loss_bbox': [], 'val_loss_total': [],
        'val_acc': [], 'val_iou': []
    }

    best_val_loss_total = float('inf')
    epochs_no_improve = 0
    best_model_state = None

    print(f"Iniciando entrenamiento por {num_epochs} épocas en dispositivo: {device}.")

    for epoch in range(num_epochs):
        epoch_start_time = time.time()
        
        # --- Fase de Entrenamiento ---
        model.train()
        running_train_loss_cls = 0.0
        running_train_loss_bbox = 0.0
        running_train_loss_total = 0.0
        correct_train_preds = 0
        total_train_samples = 0
        sum_train_iou = 0.0 # Acumulador para la suma de IoUs (no promediado aún)
        num_train_batches_with_iou = 0 # Contador para batches donde se calculó IoU

        for batch_idx, batch_data in enumerate(train_loader):
            if batch_data is None:
                continue 
            
            images, class_labels, bbox_labels_raw = batch_data
            
            images = images.to(device)
            class_labels = class_labels.to(device).long()
            
            if bbox_labels_raw.ndim == 3 and bbox_labels_raw.shape[1] == 1:
                 bbox_labels = bbox_labels_raw.to(device).float().squeeze(1)
            elif bbox_labels_raw.ndim == 2 and bbox_labels_raw.shape[-1] == 4:
                 bbox_labels = bbox_labels_raw.to(device).float()
            else:
                continue

            optimizer.zero_grad()
            class_logits, bbox_preds_raw = model(images)

            bbox_preds = torch.sigmoid(bbox_preds_raw) if apply_sigmoid_to_bbox_pred else bbox_preds_raw
            
            loss_cls = classification_criterion(class_logits, class_labels)
            loss_bbox = bbox_criterion(bbox_preds, bbox_labels) 
            total_loss = loss_cls + bbox_loss_weight * loss_bbox

            total_loss.backward()
            optimizer.step()

            batch_size = images.size(0) # Usar images.size(0) para el tamaño real del batch
            running_train_loss_cls += loss_cls.item() * batch_size
            running_train_loss_bbox += loss_bbox.item() * batch_size
            running_train_loss_total += total_loss.item() * batch_size

            _, predicted_classes = torch.max(class_logits, 1)
            correct_train_preds += (predicted_classes == class_labels).sum().item()
            total_train_samples += batch_size 
            
            # Calcula IoU solo si hay bboxes válidos en el batch para evitar errores
            if bbox_labels.numel() > 0 and bbox_preds.numel() > 0:
                 # iou_metric debería devolver la suma de IoUs para el batch o el IoU promedio del batch
                 # Si devuelve la suma de IoUs, está bien. Si devuelve el promedio, multiplica por batch_size.
                 # Asumamos que iou_metric devuelve el IoU promedio del batch.
                iou_batch_avg = iou_metric(bbox_labels.detach(), bbox_preds.detach(), apply_sigmoid_to_pred=False)
                if torch.is_tensor(iou_batch_avg) and iou_batch_avg.numel() == 1 : # Asegurarse que es un escalar
                    sum_train_iou += iou_batch_avg.item() * batch_size
                    num_train_batches_with_iou += batch_size # o simplemente num_train_batches_with_iou += 1 si iou_metric devuelve suma
                # else:
                    # print(f"Advertencia: iou_metric devolvió un valor inesperado: {iou_batch_avg}")

        print(f"DEBUG train_multitask_model: Epoch {epoch+1}, total_train_samples procesadas esta época: {total_train_samples}")

        epoch_train_loss_cls = running_train_loss_cls / total_train_samples if total_train_samples > 0 else 0.0
        epoch_train_loss_bbox = running_train_loss_bbox / total_train_samples if total_train_samples > 0 else 0.0
        epoch_train_loss_total = running_train_loss_total / total_train_samples if total_train_samples > 0 else 0.0
        epoch_train_acc = correct_train_preds / total_train_samples if total_train_samples > 0 else 0.0
        # epoch_train_iou = sum_train_iou / num_train_batches_with_iou if num_train_batches_with_iou > 0 else 0.0 # Si iou_metric devuelve promedio y contamos batches
        epoch_train_iou = sum_train_iou / total_train_samples if total_train_samples > 0 else 0.0 # Si iou_metric devuelve promedio y sum_train_iou acumula (promedio*tamaño_batch)


        history['train_loss_cls'].append(epoch_train_loss_cls)
        history['train_loss_bbox'].append(epoch_train_loss_bbox)
        history['train_loss_total'].append(epoch_train_loss_total)
        history['train_acc'].append(epoch_train_acc)
        history['train_iou'].append(epoch_train_iou)

        # --- Fase de Validación ---
        model.eval()
        running_val_loss_cls = 0.0
        running_val_loss_bbox = 0.0
        running_val_loss_total = 0.0
        correct_val_preds = 0
        total_val_samples_val = 0
        sum_val_iou = 0.0 # Acumulador para la suma de IoUs
        num_val_batches_with_iou = 0 # Contador para batches donde se calculó IoU
        current_epoch_val_loss_total = float('inf')

        if val_loader is not None and len(val_loader) > 0:
            with torch.no_grad():
                for batch_idx_val, batch_data_val in enumerate(val_loader):
                    if batch_data_val is None:
                        continue
                    
                    images_val, class_labels_val, bbox_labels_raw_val = batch_data_val

                    images_val = images_val.to(device)
                    class_labels_val = class_labels_val.to(device).long()

                    if bbox_labels_raw_val.ndim == 3 and bbox_labels_raw_val.shape[1] == 1:
                        bbox_labels_val = bbox_labels_raw_val.to(device).float().squeeze(1)
                    elif bbox_labels_raw_val.ndim == 2 and bbox_labels_raw_val.shape[-1] == 4:
                        bbox_labels_val = bbox_labels_raw_val.to(device).float()
                    else:
                        continue

                    class_logits_val, bbox_preds_raw_val = model(images_val)
                    bbox_preds_val = torch.sigmoid(bbox_preds_raw_val) if apply_sigmoid_to_bbox_pred else bbox_preds_raw_val

                    loss_cls_val = classification_criterion(class_logits_val, class_labels_val)
                    loss_bbox_val = bbox_criterion(bbox_preds_val, bbox_labels_val)
                    total_loss_val = loss_cls_val + bbox_loss_weight * loss_bbox_val
                    
                    batch_size_val = images_val.size(0)
                    running_val_loss_cls += loss_cls_val.item() * batch_size_val
                    running_val_loss_bbox += loss_bbox_val.item() * batch_size_val
                    running_val_loss_total += total_loss_val.item() * batch_size_val

                    _, predicted_classes_val = torch.max(class_logits_val, 1)
                    correct_val_preds += (predicted_classes_val == class_labels_val).sum().item()
                    total_val_samples_val += batch_size_val
                    
                    if bbox_labels_val.numel() > 0 and bbox_preds_val.numel() > 0:
                        iou_batch_avg_val = iou_metric(bbox_labels_val.detach(), bbox_preds_val.detach(), apply_sigmoid_to_pred=False)
                        if torch.is_tensor(iou_batch_avg_val) and iou_batch_avg_val.numel() == 1:
                            sum_val_iou += iou_batch_avg_val.item() * batch_size_val
                            num_val_batches_with_iou += batch_size_val
            
            epoch_val_loss_cls = running_val_loss_cls / total_val_samples_val if total_val_samples_val > 0 else 0.0
            epoch_val_loss_bbox = running_val_loss_bbox / total_val_samples_val if total_val_samples_val > 0 else 0.0
            current_epoch_val_loss_total = running_val_loss_total / total_val_samples_val if total_val_samples_val > 0 else float('inf')
            epoch_val_acc = correct_val_preds / total_val_samples_val if total_val_samples_val > 0 else 0.0
            # epoch_val_iou = sum_val_iou / num_val_batches_with_iou if num_val_batches_with_iou > 0 else 0.0
            epoch_val_iou = sum_val_iou / total_val_samples_val if total_val_samples_val > 0 else 0.0


        else: 
            epoch_val_loss_cls, epoch_val_loss_bbox = 0.0, 0.0
            current_epoch_val_loss_total = float('inf') 
            epoch_val_acc, epoch_val_iou = 0.0, 0.0
            total_val_samples_val = 0

        history['val_loss_cls'].append(epoch_val_loss_cls)
        history['val_loss_bbox'].append(epoch_val_loss_bbox)
        history['val_loss_total'].append(current_epoch_val_loss_total)
        history['val_acc'].append(epoch_val_acc)
        history['val_iou'].append(epoch_val_iou)

        epoch_duration = time.time() - epoch_start_time
        print(f"Epoch {epoch+1}/{num_epochs} Duración: {epoch_duration:.2f}s")
        print(f"  Train: LossCls={epoch_train_loss_cls:.4f}, LossBbox={epoch_train_loss_bbox:.4f}, Total={epoch_train_loss_total:.4f}, Acc={epoch_train_acc:.4f}, IoU={epoch_train_iou:.4f}")
        
        if total_val_samples_val > 0:
            print(f"  Valid: LossCls={epoch_val_loss_cls:.4f}, LossBbox={epoch_val_loss_bbox:.4f}, Total={current_epoch_val_loss_total:.4f}, Acc={epoch_val_acc:.4f}, IoU={epoch_val_iou:.4f}")
        else:
            print("  Valid: No hay datos de validación para esta época.")

        # 📌 Lógica de Scheduler (si se proporciona)
        if scheduler is not None:
            if isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                if total_val_samples_val > 0: 
                    scheduler.step(current_epoch_val_loss_total) # 📌 LLAMADA AL SCHEDULER
                # else: Opcionalmente, si no hay validación, no hacer step o basarse en train_loss
            else: # Para otros schedulers que hacen step por época sin argumentos
                scheduler.step()
            
            # Opcional: imprimir la tasa de aprendizaje actual
            current_lr = optimizer.param_groups[0]['lr']
            print(f"  Current LR: {current_lr:.1e}")


        # Lógica de Early Stopping y guardado del mejor modelo
        if total_val_samples_val > 0: 
            if current_epoch_val_loss_total < best_val_loss_total:
                best_val_loss_total = current_epoch_val_loss_total
                if model is not None:
                    best_model_state = copy.deepcopy(model.state_dict())
                    try:
                        torch.save(best_model_state, model_save_path)
                        print(f"  Mejor modelo guardado en '{model_save_path}' con val_loss_total: {best_val_loss_total:.4f}")
                    except Exception as e_save:
                        print(f"ERROR AL INTENTAR GUARDAR EL MODELO en '{model_save_path}': {e_save}")
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1
                print(f"  Val_loss_total no mejoró. Paciencia: {epochs_no_improve}/{patience}")
            
            if epochs_no_improve >= patience:
                print(f"Early stopping activado después de {epoch+1} épocas.")
                break
        elif epoch == 0 and best_model_state is None : 
             if model is not None:
                best_model_state = copy.deepcopy(model.state_dict())
                try:
                    torch.save(best_model_state, model_save_path)
                    print(f"  Modelo de la primera época guardado (sin validación activa) en '{model_save_path}'")
                except Exception as e_save:
                    print(f"ERROR AL INTENTAR GUARDAR EL MODELO (primera época sin val) en '{model_save_path}': {e_save}")

    if best_model_state:
        print("Cargando el mejor estado del modelo guardado al final del entrenamiento.")
        model.load_state_dict(best_model_state)
    else:
        print("No se guardó ningún 'mejor modelo'. El modelo tiene los pesos de la última época entrenada.")
    
    print("Entrenamiento finalizado.")
    return model, history

## Run

In [36]:
#  DEFINICIÓN DE collate_fn_skip_none (colócala aquí o en una celda anterior si ya está)
def collate_fn_skip_none(batch):
    batch_original_len = len(batch)
    batch_filtered = [item for item in batch if item is not None]
    if batch_original_len != len(batch_filtered) and batch_original_len > 0:
        pass # print(f"DEBUG collate_fn: Filtradas {batch_original_len - len(batch_filtered)} muestras None.")
    if not batch_filtered:
        return None
    try:
        return torch.utils.data.dataloader.default_collate(batch_filtered)
    except Exception as e:
        # print(f"ERROR en default_collate dentro de collate_fn_skip_none: {e}")
        raise

# if __name__ == '__main__': # Descomenta si ejecutas como script .py

reset_seed(42) 

DATA_DIR_RUN = '../Datos/' 
BATCH_SIZE_RUN = 32 

# DIR_Results DEBE estar definida globalmente (ej. DIR_Results = './Resultados/')
if 'DIR_Results' not in globals():
    print("ADVERTENCIA: DIR_Results no estaba definido globalmente. Usando './Resultados_default/' para esta ejecución.")
    DIR_Results = './Resultados_default/' 

# Convertir DIR_Results a una ruta absoluta y crearla
absolute_dir_results = os.path.abspath(DIR_Results)
os.makedirs(absolute_dir_results, exist_ok=True) 
print(f"Los resultados se guardarán en el directorio: {absolute_dir_results}")


print("Cargando y preprocesando el DataFrame...")
img_dir_run = osp.join(DATA_DIR_RUN, "images/images")


if not osp.exists(img_dir_run):
    print(f"ERROR: El directorio de imágenes no existe: {img_dir_run}")
path_train_csv = osp.join(DATA_DIR_RUN, "train.csv")
if not osp.exists(path_train_csv):
    print(f"ERROR: El archivo 'train.csv' no se encontró en {DATA_DIR_RUN}")
try:
    df_original_run = pd.read_csv(path_train_csv)
except Exception as e:
    print(f"ERROR: No se pudo leer 'train.csv': {e}")
df_original_run['class_id'] = df_original_run['class'].map(obj2id) 
df_original_run.dropna(subset=['class_id'], inplace=True)
df_original_run['class_id'] = df_original_run['class_id'].astype(int)
columns_f_run = ['filename', 'xmin', 'ymin', 'xmax', 'ymax', 'class', 'class_id']
missing_cols = [col for col in columns_f_run if col not in df_original_run.columns]
if missing_cols:
    print(f"ERROR: Faltan las siguientes columnas en train.csv: {missing_cols}.")
df_run = df_original_run[columns_f_run].copy()
if not df_run.empty:
    bbox_cols_run = ['xmin', 'ymin', 'xmax', 'ymax']
    for col in bbox_cols_run:
        df_run[col] = pd.to_numeric(df_run[col], errors='coerce')
    df_run.dropna(subset=bbox_cols_run, inplace=True)
    if not df_run.empty:
        df_run['xmin'] = df_run['xmin'] / IMG_ORIG_W
        df_run['xmax'] = df_run['xmax'] / IMG_ORIG_W
        df_run['ymin'] = df_run['ymin'] / IMG_ORIG_H
        df_run['ymax'] = df_run['ymax'] / IMG_ORIG_H
        df_run[['xmin', 'ymin', 'xmax', 'ymax']] = df_run[['xmin', 'ymin', 'xmax', 'ymax']].clip(0.0, 1.0)
        print("Bounding boxes en el script de Run normalizados y acotados a [0,1].")
    else:
        print("ERROR: DataFrame para el script de Run vacío después de asegurar columnas de bbox numéricas.")
else:
    print("ERROR: DataFrame para el script de Run está vacío inicialmente.")
if df_run.empty or 'class_id' not in df_run.columns or df_run['class_id'].isnull().all():
    print("ERROR: DataFrame vacío o sin 'class_id' válidos para la partición estratificada.")
    train_df_run, val_df_run = pd.DataFrame(columns=df_run.columns), pd.DataFrame(columns=df_run.columns) 
else:
    can_stratify = True
    if df_run['class_id'].nunique() < 2 : 
        can_stratify = False
    else:
        value_counts = df_run['class_id'].value_counts()
        if any(value_counts < 2): 
            can_stratify = False
    if can_stratify:
        try:
            train_df_run, val_df_run = train_test_split(df_run, test_size=0.2, random_state=42, stratify=df_run['class_id'])
        except ValueError as e: 
            print(f"Advertencia: Error durante la partición estratificada ({e}). Usando partición no estratificada.")
            train_df_run, val_df_run = train_test_split(df_run, test_size=0.2, random_state=42)
    else:
        print("Advertencia: No hay suficientes muestras/clases para una partición estratificada. Usando partición no estratificada.")
        train_df_run, val_df_run = train_test_split(df_run, test_size=0.2, random_state=42)
print(f"DEBUG: Longitud de df_run: {len(df_run)}")
print(f"DEBUG: Longitud de train_df_run: {len(train_df_run)}")


print(f"Usando medias de ImageNet para el script de Run: {MEANS_IMAGENET}")
print(f"Usando desviaciones estándar de ImageNet para el script de Run: {STDS_IMAGENET}")

print("DEBUG: Usando pipeline de transformación de entrenamiento MÍNIMO con BboxParams por DEFECTO.")
run_train_transforms = A.Compose([
    A.Resize(height=TARGET_H, width=TARGET_W), 
    #A.HorizontalFlip(p=0.5),
    #A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.3),
    A.RandomBrightnessContrast(brightness_limit=0.05, contrast_limit=0.05, p=0.2), # <<< MUY SUAVE y p baja
    #A.ShiftScaleRotate(shift_limit=0.02, scale_limit=0.02, rotate_limit=5, p=0.2, border_mode=cv2.BORDER_CONSTANT), # 'value' eliminado; si da warning, comenta la línea.
    A.Normalize(mean=MEANS_IMAGENET, std=STDS_IMAGENET, max_pixel_value=255.0),
    ToTensorV2() 
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

run_eval_transforms = A.Compose([ 
    A.Resize(height=TARGET_H, width=TARGET_W),
    A.Normalize(mean=MEANS_IMAGENET, std=STDS_IMAGENET, max_pixel_value=255.0),
    ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

print("Creando Datasets y DataLoaders finales para el script de Run...")
# ... (Creación de train_dataset_run, val_dataset_run como antes) ...
if not train_df_run.empty:
    train_dataset_run = militarDataset(df=train_df_run, img_dir=img_dir_run, transform=run_train_transforms)
else:
    train_dataset_run = [] 
    print("Advertencia: train_df_run está vacío, train_dataset_run será una lista vacía.")
if not val_df_run.empty:
    val_dataset_run = militarDataset(df=val_df_run, img_dir=img_dir_run, transform=run_eval_transforms)
else:
    val_dataset_run = []
    print("Advertencia: val_df_run está vacío, val_dataset_run será una lista vacía.")

# ... (Bucle de inspección de train_dataset_run como antes) ...
if len(train_dataset_run) > 0:
    print(f"\nDEBUG: Inspeccionando las primeras (hasta) 5 muestras de train_dataset_run (longitud total: {len(train_dataset_run)}):")
    valid_samples_count_debug = 0
    for i in range(min(5, len(train_dataset_run))):
        # print(f"  DEBUG: Intentando obtener train_dataset_run[{i}]...") # Puede ser muy verboso
        try:
            sample = train_dataset_run[i] 
            if sample is None:
                # print(f"    DEBUG: train_dataset_run[{i}] devolvió None.")
                pass
            else:
                # img_tensor, cls_tensor, bbox_tensor = sample
                # print(f"    DEBUG: train_dataset_run[{i}] OK. img_shape={img_tensor.shape}, cls_id={cls_tensor.item()}, bbox={bbox_tensor.squeeze().tolist()}")
                valid_samples_count_debug += 1
        except Exception as e:
            # print(f"    DEBUG: Error al obtener train_dataset_run[{i}]: {e}")
            pass
    print(f"DEBUG: De las primeras muestras inspeccionadas, {valid_samples_count_debug} fueron válidas.\n")
    if len(train_df_run) > 0 and valid_samples_count_debug == 0 and len(train_dataset_run) > 0: 
        print("ALERTA DE DEBUG: Ninguna de las primeras muestras del dataset de entrenamiento es válida. Revisa los prints internos de 'militarDataset'.")
else:
    print("DEBUG: train_dataset_run está vacío o es una lista vacía, no se pueden inspeccionar muestras.")


# ... (Creación de DataLoaders como antes) ...
if len(train_dataset_run) > 0: 
    train_loader_run = DataLoader(train_dataset_run, batch_size=BATCH_SIZE_RUN, shuffle=True,
                                  num_workers=0, generator=generator, collate_fn=collate_fn_skip_none)
else:
    train_loader_run = None 
    print("Advertencia: El DataLoader de entrenamiento (train_loader_run) no se creará porque el dataset está vacío.")
if len(val_dataset_run) > 0:
    val_loader_run = DataLoader(val_dataset_run, batch_size=BATCH_SIZE_RUN, shuffle=False,
                                num_workers=0, generator=generator, collate_fn=collate_fn_skip_none)
else:
    val_loader_run = None
    print("Advertencia: El DataLoader de validación (val_loader_run) no se creará porque el dataset está vacío.")

device_run = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo para el script de Run: {device_run}")


model_run = ResNet50MultiTaskModel(num_classes=NUM_CLASSES, 
                                   freeze_backbone=True, 
                                   use_deeper_bbox_head=False).to(device_run)
print("Usando cabeza de regresión de BBox MÁS PROFUNDA.")


classification_criterion_run = nn.CrossEntropyLoss() #(Probado: )
bbox_criterion_run = nn.SmoothL1Loss() # 📌 AJUSTA AQUÍ (Probado: nn.L1Loss(); nn.SmoothL1Loss()) #Probar:  


params_to_train_run = [param for param in model_run.parameters() if param.requires_grad]
if not params_to_train_run:
    print("ERROR: No hay parámetros para entrenar. Verifica 'freeze_backbone'.")
    optimizer_run = None
    scheduler_run = None 
else:
    optimizer_run = optim.Adam(params_to_train_run, lr=1e-3) # O el lr que estés usando
    scheduler_run = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer_run, 
        mode='min',     
        factor=0.1,     
        patience=7,     # Paciencia del scheduler
        min_lr=1e-6     
    )


# Generar nombre de archivo con fecha y hora y usar RUTA ABSOLUTA ---
ahora = datetime.now() 
timestamp_str = ahora.strftime("%Y%m%d_%H%M") 
model_filename = f"pretrained_model_{timestamp_str}.pth"

model_save_path_run = osp.join(absolute_dir_results, model_filename) 

# --- Generar nombre de archivo para guardar el modelo de ESTA EJECUCIÓN ---
timestamp_str_run = ahora.strftime("%Y%m%d_%H%M%S") # Segundos para mayor unicidad
model_filename_run = f"pretrained_model_{timestamp_str_run}.pth"
model_save_path_run = osp.join(DIR_Results, model_filename_run) 
print(f"DEBUG: El modelo se guardará en la ruta absoluta: {model_save_path_run}")


print("Iniciando el proceso de entrenamiento del script de Run...")
APPLY_SIGMOID_FLAG_RUN = True 
print(f"APPLY_SIGMOID_TO_BBOX_PRED para el script de Run está configurado a: {APPLY_SIGMOID_FLAG_RUN}")

if train_loader_run is not None and optimizer_run is not None: 
    trained_model_run, history_run = train_multitask_model(
        model=model_run,
        train_loader=train_loader_run,
        val_loader=val_loader_run, 
        classification_criterion=classification_criterion_run,
        bbox_criterion=bbox_criterion_run,
        optimizer=optimizer_run,
        device=device_run,
        num_epochs=120, # 📌 AJUSTA AQUÍ 
        bbox_loss_weight=20.0, # 📌 AJUSTA AQUÍ 
        patience=15, # 📌 AJUSTA AQUÍ 
        model_save_path=model_save_path_run,
        apply_sigmoid_to_bbox_pred=APPLY_SIGMOID_FLAG_RUN
    )
    print("Entrenamiento del script de Run completado.")
else:
    print("No se ejecutó el entrenamiento del script de Run porque el train_loader_run está vacío/None o no hay parámetros para entrenar.")


Los resultados se guardarán en el directorio: c:\Users\allc0363\OneDrive\MCD - Icesi\Semestre 2\Fundamentos 2\Taller 1\FAII_TALLER1\Notebooks\Resultados
Cargando y preprocesando el DataFrame...
Bounding boxes en el script de Run normalizados y acotados a [0,1].
DEBUG: Longitud de df_run: 189
DEBUG: Longitud de train_df_run: 151
Usando medias de ImageNet para el script de Run: [0.485, 0.456, 0.406]
Usando desviaciones estándar de ImageNet para el script de Run: [0.229, 0.224, 0.225]
DEBUG: Usando pipeline de transformación de entrenamiento MÍNIMO con BboxParams por DEFECTO.
Creando Datasets y DataLoaders finales para el script de Run...
DEBUG militarDataset: Inicializado con 151 filas. Transformaciones: Sí. Labeled: True
DEBUG militarDataset: Inicializado con 38 filas. Transformaciones: Sí. Labeled: True

DEBUG: Inspeccionando las primeras (hasta) 5 muestras de train_dataset_run (longitud total: 151):
DEBUG: De las primeras muestras inspeccionadas, 5 fueron válidas.

Usando dispositivo 